https://docs.opencv.org/3.1.0/da/d22/tutorial_py_canny.html

https://www.meccanismocomplesso.org/en/opencv-python-canny-edge-detection/

https://pythonprogramming.net/template-matching-python-opencv-tutorial/?completed=/canny-edge-detection-gradients-python-opencv-tutorial/

image gradients and edge detection. Image gradients can be used to measure directional intensity. 
We can use these gradients to convert to pure edges, can also use [Canny Edge detection](https://docs.opencv.org/master/dd/d1a/group__imgproc__feature.html#ga04723e007ed888ddf11d9ba04e2232de).

In [3]:
import cv2
import numpy as np

frame = cv2.imread('img11.jpg',1)
frame = cv2.resize(frame, dsize=None, fx=0.3, fy=0.3)
hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

lower_red = np.array([30,150,50])
upper_red = np.array([255,255,180])

mask = cv2.inRange(hsv, lower_red, upper_red)
res = cv2.bitwise_and(frame,frame, mask= mask)

kernel = np.ones((5,5),np.uint8)
erosion = cv2.erode(mask,kernel,iterations=1)
dilation = cv2.dilate(mask,kernel,iterations=1)

opening = cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel)
closing = cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)

laplacian = cv2.Laplacian(frame,cv2.CV_64F)
sobelx = cv2.Sobel(frame,cv2.CV_64F,1,0,ksize=5)
sobely = cv2.Sobel(frame,cv2.CV_64F,0,1,ksize=5)

while True:
    cv2.imshow('Original',frame)
    # cv2.imshow('Mask',res)
    # cv2.imshow('erosion',erosion)
    # cv2.imshow('dilation',dilation)
    # cv2.imshow('opening',opening)
    # cv2.imshow('closing',closing)
    cv2.imshow('laplacian',laplacian)
    cv2.imshow('sobelx',sobelx)
    cv2.imshow('sobely',sobely)

    k = cv2.waitKey(0)
    if k == ord('q'):
        break

cv2.destroyAllWindows()

In [4]:
# canny边缘检测：1.灰度处理，2.高斯滤波(去除噪声)，3.调用canny方法

import cv2
import numpy as np

frame = cv2.imread('img11.jpg',1)
frame = cv2.resize(frame, dsize=None, fx=0.3, fy=0.3)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# 使用高斯模糊消除高频噪声，3x3内核
imgG = cv2.GaussianBlur(gray,(3,3),0)
# 查找图像边缘，图片某处经过卷积后的值与阈值进行比较
edges = cv2.Canny(imgG,20,100) #img, lower, upper

while True:
    cv2.imshow('Original',frame)
    cv2.imshow('Canny',edges)

    k = cv2.waitKey(0)
    if k == ord('q'):
        break

cv2.destroyAllWindows()

## Zero-parameter, automatic Canny edge detection

it’s clear that the automatic, zero-parameter version of the Canny edge detection obtains the best results with the least effort.

[zero-parameter-automatic-canny-edge-detection-with-python-and-opencv](https://www.pyimagesearch.com/2015/04/06/zero-parameter-automatic-canny-edge-detection-with-python-and-opencv/)，auto_canny.py

### The Canny edge detector

The Canny edge detector is still widely used today was one of the default edge detectors in image processing.

The Canny edge detection algorithm can be broken down into 5 steps:

1. Smooth the image using a Gaussian filter to remove high frequency noise.
2. Compute the gradient intensity representations of the image.
3. Apply non-maximum suppression to remove “false” responses to to edge detection.
4. Apply thresholding using a lower and upper boundary on the gradient values.
5. Track edges using hysteresis by suppressing weak edges that are not connected to strong edges.

The OpenCV implementation of the Canny edge detector: `cv2.canny(image, lower, upper)`.

The lower and upper are integer thresholds. The problem becomes determining optimal lower and upper threshold values.

It is important when processing multiple images with different contents captured under varying lighting conditions.

This is a trick that relies on basic statistics that can remove the manual tuning of the thresholds for Canny edge detection. This trick will save time for parameter tuning — and still get a nice Canny edge map after applying the function.

In [5]:
import numpy as np
import argparse # 解析命令行参数
import glob # 获得图像的磁盘路径
import cv2

# image是单通道图像
# sigma可以用来改变基于简单统计数据确定的百分比阈值
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # construct two thresholds using the median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    # 边缘检测,灰度值小于lower的会被丢弃，大于upper的会被保留，之间的部分自动检测
    edged = cv2.Canny(image, lower, upper)
    print("lower:", lower, "upper:", upper)

    # return the edged image
    return edged

In [8]:
# load the image, convert it to grayscale, and blur it slightly
image = cv2.imread('img11.jpg')
image = cv2.resize(image, dsize=None, fx=0.3, fy=0.3)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3, 3), 0)

# apply Canny edge detection
wide = cv2.Canny(blurred, 10, 200) # A wide threshold
tight = cv2.Canny(blurred, 225, 250) # A tight threshold
auto = auto_canny(blurred) # automatically determined threshold

# show the images
while True:
    cv2.imshow("Original", image)
    cv2.imshow("Edges", np.hstack([wide, tight, auto]))
    k = cv2.waitKey(0)
    if k==ord('q'):
        break

cv2.destroyAllWindows()

lower: 53 upper: 106
